In [9]:
import numpy as np

# 1. Load Data (Adjust path to where you uploaded the files)
# usually named 'Train_Dst_NoAuction_ZScore_CF_7.txt' etc.
try:
    # Example loading one day of data to test
    data = np.loadtxt('data/Train_Dst_NoAuction_DecPre_CF_7.txt')
    data = data.T
    print("✅ Data Loaded Successfully!")
    print(f"Shape: {data.shape}") # Should be (Number_of_Events, 149)
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you dragged the 'data' folder into the right place!")

✅ Data Loaded Successfully!
Shape: (254750, 149)


In [10]:
import torch
import torch.nn as nn

class DeepLOB(nn.Module):
    def __init__(self, y_len):
        super().__init__()
        self.y_len = y_len
        
        # 1. Convolutional Block
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
        )
        
        # 2. Inception Module
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
        )
        
        # 3. LSTM (The "Memory")
        # CHANGED: input_size=32 -> input_size=320
        # Why? The Conv output is (Batch, 32 channels, Time, 10 features). 32 * 10 = 320.
        self.lstm = nn.LSTM(input_size=320, hidden_size=64, num_layers=1, batch_first=True)
        
        # 4. Output Layer
        self.fc1 = nn.Linear(64, self.y_len)

    def forward(self, x):
        # x shape: (Batch, 1, 100, 40)
        x = self.conv1(x)
        x = self.conv2(x)
        
        # Reshape for LSTM
        # Current shape: (Batch, 32, Time, 10)
        x = x.permute(0, 2, 1, 3) # -> (Batch, Time, 32, 10)
        x = x.reshape(x.shape[0], x.shape[1], -1) # -> (Batch, Time, 320)
        
        x, _ = self.lstm(x)
        x = x[:, -1, :] # Take last time step
        forecast = self.fc1(x)
        return forecast

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# ==========================================
# 1. HELPER: PREPARE DATA FOR CNN
# ==========================================
def prepare_x_y(data, k, T=100):
    # data shape: (N, 149) -> last 5 cols are labels, first 144 are features
    # We only need the first 40 features (10 levels * (Price+Vol) * (Bid+Ask))
    
    # Extract features (first 40 columns) & labels (target horizon k)
    # k=0 -> 10 tick horizon, k=1 -> 20 tick, etc. (Check dataset readme)
    # Here we just use column -5 as a simple target for testing
    
    x = data[:, :40] 
    y = data[:, -5] - 1 # Labels are 1,2,3 -> convert to 0,1,2 for PyTorch
    
    X, Y = [], []
    
    # Sliding Window: take T past events to predict next label
    # We'll just take the first 5000 samples to make it fast for testing!
    limit = 5000 
    
    for i in range(T, min(len(data), limit)):
        X.append(x[i-T:i])
        Y.append(y[i-1]) # Label corresponding to the end of the window
    
    return np.array(X), np.array(Y)

print("⏳ Preparing data (reshaping to 100x40 images)...")
# k=1 means we are predicting the "Next 20 Ticks" direction
X_train, y_train = prepare_x_y(data, k=1) 

# Convert to PyTorch Tensors
# CNN Input Shape: (Batch, 1, 100, 40) -> (Batch, Channel, Time, Features)
X_tensor = torch.from_numpy(X_train).float().unsqueeze(1) 
y_tensor = torch.from_numpy(y_train).long()

# Create DataLoader (Batches)
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)
print(f"✅ Data Ready! Created {len(train_loader)} batches.")

# ==========================================
# 2. TRAINING LOOP
# ==========================================
# Initialize Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepLOB(y_len=3).to(device) # Make sure this matches your Class definition
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"🚀 Starting Training on {device}...")
model.train()

epochs = 5
for epoch in range(epochs):
    total_loss = 0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        # Move data to GPU (if available)
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 1. Zero gradients
        optimizer.zero_grad()
        
        # 2. Forward pass (Prediction)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 3. Backward pass (Learning)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate Accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f} | Acc: {100 * correct / total:.2f}%")

print("🎉 CONGRATULATIONS! You just trained a Deep Learning HFT Model.")

⏳ Preparing data (reshaping to 100x40 images)...
✅ Data Ready! Created 307 batches.
🚀 Starting Training on cpu...


Epoch 1/5 | Loss: 1.0799 | Acc: 43.08%
Epoch 2/5 | Loss: 1.0784 | Acc: 43.27%
Epoch 3/5 | Loss: 1.0791 | Acc: 43.27%
Epoch 4/5 | Loss: 1.0779 | Acc: 43.27%
Epoch 5/5 | Loss: 1.0775 | Acc: 43.27%
🎉 CONGRATULATIONS! You just trained a Deep Learning HFT Model.


In [12]:
# Save the trained weights to a file
torch.save(model.state_dict(), 'deeplob_weights.pth')
print("✅ Brain saved to 'deeplob_weights.pth'. Download this file!")

✅ Brain saved to 'deeplob_weights.pth'. Download this file!
